In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install scikit-image

from skimage.color import lab2rgb
import sys


# representative RGB colours for each label, for nice display
COLOUR_RGB = {
    'red': (255, 0, 0),
    'orange': (255, 114, 0),
    'yellow': (255, 255, 0),
    'green': (0, 230, 0),
    'blue': (0, 0, 255),
    'purple': (187, 0, 187),
    'brown': (117, 60, 0),
    'pink': (255, 187, 187),
    'black': (0, 0, 0),
    'grey': (150, 150, 150),
    'white': (255, 255, 255),
}
name_to_rgb = np.vectorize(COLOUR_RGB.get, otypes=[np.uint8, np.uint8, np.uint8])


def plot_predictions(model, lum=71, resolution=256):
    """
    Create a slice of LAB colour space with given luminance; predict with the model; plot the results.
    """
    wid = resolution
    hei = resolution
    n_ticks = 5

    # create a hei*wid grid of LAB colour values, with L=lum
    ag = np.linspace(-100, 100, wid)
    bg = np.linspace(-100, 100, hei)
    aa, bb = np.meshgrid(ag, bg)
    ll = lum * np.ones((hei, wid))
    lab_grid = np.stack([ll, aa, bb], axis=2)

    # convert to RGB for consistency with original input
    X_grid = lab2rgb(lab_grid)

    # predict and convert predictions to colours so we can see what's happening
    y_grid = model.predict(X_grid.reshape((wid*hei, 3)))
    pixels = np.stack(name_to_rgb(y_grid), axis=1) / 255
    pixels = pixels.reshape((hei, wid, 3))

    # plot input and predictions
    plt.figure(figsize=(10, 5))
    plt.suptitle('Predictions at L=%g' % (lum,))
    plt.subplot(1, 2, 1)
    plt.title('Inputs')
    plt.xticks(np.linspace(0, wid, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.yticks(np.linspace(0, hei, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.xlabel('A')
    plt.ylabel('B')
    plt.imshow(X_grid.reshape((hei, wid, 3)))

    plt.subplot(1, 2, 2)
    plt.title('Predicted Labels')
    plt.xticks(np.linspace(0, wid, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.yticks(np.linspace(0, hei, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.xlabel('A')
    plt.imshow(pixels)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [29]:
import pandas as pd

# load data set
file_path = "colour-data.csv"
df = pd.read_csv(file_path)

# shows data set
df.head()


,R,G,B,Label,Confidence
0,168,211,243,blue,good
1,37,32,40,black,perfect
2,35,34,38,black,perfect
3,79,74,159,purple,good
4,55,99,34,green,perfect


In [30]:
import numpy as np


X = df[['R', 'G', 'B']].values / 255.0


y = df['Label'].values

# checks shapes of x and y
X.shape, y.shape


((3950, 3), (3950,))

In [33]:
from sklearn.model_selection import train_test_split

# random state
random_state = 42

# 70/30 split
X_train, X_remainder, y_train, y_remainder = train_test_split(X, y, test_size=0.3, random_state=random_state)

# 15/15 split
X_valid, X_test, y_valid, y_test = train_test_split(X_remainder, y_remainder, test_size=0.5, random_state=random_state)

# check sizes
X_train.shape, X_valid.shape, X_test.shape


((2765, 3), (592, 3), (593, 3))

In [ ]:
from sklearn.linear_model import LogisticRegression

# created model called model_rgb
model_rgb = LogisticRegression(max_iter=500, random_state=random_state)
model_rgb.fit(X_train, y_train)

# evaluation
validation_score_rgb = model_rgb.score(X_valid, y_valid)
validation_score_rgb
print("Validation Score (RGB Model):", validation_score_rgb)


Validation Score (RGB Model): 0.6925675675675675
